## 데이터 로드
- .env파일 생성필요
- API_KEY = "your OpenAPI example"

In [1]:
import pandas as pd
import json
import os


from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [2]:
load_dotenv()

api_key = os.getenv('API_KEY')

df = pd.read_csv("../data/Fine-tuning Data_raw.csv").drop(columns="텍스트 - ERD 생성 데이터").drop(columns= '인덱스')
dataset = df.copy()
dataset.columns = ['text']
dataset

,text
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안..."
...,...
95,"지역 축제의 혼잡도를 낮추고 운영 효율을 높이기 위해, 관람객의 이동 경로와 체류 ..."
96,작물의 생장 조건을 분석하고 환경을 자동으로 조절하는 스마트팜 제어 시스템을 기획하...
97,청소년의 감정 상태를 분석하고 적절한 상담 자원을 연결해주는 정서 관리 플랫폼을 기...
98,가정에서 발생하는 음식물 쓰레기를 줄이기 위한 스마트 분리 수거 시스템을 기획하였다...


## 데이터 작성을 위한 프롬포트 작성

In [3]:
def contents_data(df):

  contents_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 체계적으로 분석하고 요약해주는 프로그램입니다.
  주어진 텍스트를 핵심 요소별로 정리하여 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
    "project_summary": {{
      "title": "사용자 관리 플랫폼",
      "category": "웹 애플리케이션",
      "platform_type": "웹/모바일",
      "target_users": [
        "일반 사용자",
        "관리자",
        "기업 고객"
      ],
      "main_purpose": "효율적인 사용자 관리 및 데이터 분석을 통한 비즈니스 최적화",
      "key_features": [
        {{
          "feature": "사용자 등록 및 인증",
          "description": "안전한 회원가입, 로그인, 비밀번호 관리 기능"
        }},
        {{
          "feature": "대시보드",
          "description": "사용자 활동 통계 및 데이터 시각화"
        }},
        {{
          "feature": "권한 관리",
          "description": "역할 기반 접근 제어 시스템"
        }}
      ],
      "core_technologies": [
        {{
          "category": "Frontend",
          "technologies": ["React", "TypeScript", "Tailwind CSS"]
        }},
        {{
          "category": "Backend", 
          "technologies": ["Node.js", "Express", "JWT"]
        }},
        {{
          "category": "Database",
          "technologies": ["PostgreSQL", "Redis"]
        }},
        {{
          "category": "Infrastructure",
          "technologies": ["AWS", "Docker", "Nginx"]
        }}
      ],
      "problem_solving": {{
        "current_problem": "기존 사용자 관리 시스템의 비효율성과 보안 취약점",
        "solution_approach": "모던 웹 기술과 AI 기반 분석을 통한 통합 관리 플랫폼 구축",
        "expected_benefits": [
          "관리 효율성 50% 향상",
          "보안 강화를 통한 데이터 보호",
          "사용자 경험 개선"
        ]
      }},
      "special_features": [
        "AI 기반 사용자 행동 분석",
        "실시간 알림 시스템",
        "다국어 지원",
        "모바일 반응형 디자인"
      ],
      "business_model": {{
        "type": "SaaS",
        "revenue_streams": [
          "월 구독료",
          "프리미엄 기능",
          "기업용 라이선스"
        ],
        "target_market": "중소기업 및 스타트업"
      }},
      "scalability": {{
        "user_capacity": "최대 10만명 동시 접속",
        "expansion_plan": "글로벌 서비스 확장",
        "integration_capability": "타사 시스템 연동 API 제공"
      }},
      "development_timeline": {{
        "estimated_duration": "6개월",
        "key_milestones": [
          {{
            "phase": "기획 및 설계",
            "duration": "1개월"
          }},
          {{
            "phase": "MVP 개발",
            "duration": "3개월"
          }},
          {{
            "phase": "테스트 및 배포",
            "duration": "2개월"
          }}
        ]
      }},
      "success_metrics": [
        "사용자 증가율",
        "시스템 응답 시간",
        "사용자 만족도",
        "보안 사고 발생률"
      ],
      "challenges_and_risks": [
        {{
          "challenge": "데이터 보안",
          "mitigation": "다층 보안 체계 구축"
        }},
        {{
          "challenge": "확장성",
          "mitigation": "마이크로서비스 아키텍처 적용"
        }}
      ]
    }}
  }}

  다음 항목들을 반드시 포함하여 요약하세요:
  - 프로젝트 제목 및 카테고리
  - 타겟 사용자 및 주요 목적
  - 핵심 기능들과 상세 설명
  - 사용 기술 스택 (명시된 경우)
  - 해결하고자 하는 문제와 솔루션
  - 특별한 기능이나 차별점
  - 비즈니스 모델 (추론 가능한 경우)
  - 확장성 및 향후 계획
  - 예상 개발 일정 (추정)
  - 성공 지표 및 도전 과제

  텍스트에서 명시되지 않은 부분은 프로젝트 성격을 고려하여 합리적으로 추론하여 작성하세요.
  각 항목별로 구체적이고 실용적인 정보를 제공하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    data = json.loads(answer.content)
    contents_lst.append(data)
    print(f'내용 요약의 {i}번 진행중입니다.')
  return contents_lst 

def API_specs_data(df):

  API_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 기반으로 예상되는 API 명세서를 작성해주는 프로그램입니다.
  주제에 대해서 예상되는 RESTful API 명세서를 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
  "api_specification": {{
      "openapi": "3.0.0",
      "info": {{
      "title": "User Management API",
      "version": "1.0.0",
      "description": "사용자 관리를 위한 API"
      }},
      "servers": [
      {{
          "url": "https://api.example.com/v1",
          "description": "Production server"
      }}
      ],
      "paths": {{
      "/users": {{
          "get": {{
          "summary": "사용자 목록 조회",
          "description": "등록된 모든 사용자 목록을 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "page",
              "in": "query",
              "description": "페이지 번호",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 1
              }}
              }},
              {{
              "name": "limit",
              "in": "query", 
              "description": "페이지당 항목 수",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 10
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 목록 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "type": "array",
                          "items": {{
                          "$ref": "#/components/schemas/User"
                          }}
                      }},
                      "pagination": {{
                          "$ref": "#/components/schemas/Pagination"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "post": {{
          "summary": "새 사용자 생성",
          "description": "새로운 사용자를 생성합니다",
          "tags": ["Users"],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/CreateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "201": {{
              "description": "사용자 생성 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청 데이터",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }},
      "/users/{{userId}}": {{
          "get": {{
          "summary": "특정 사용자 조회",
          "description": "사용자 ID로 특정 사용자 정보를 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "put": {{
          "summary": "사용자 정보 수정",
          "description": "사용자 정보를 수정합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/UpdateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "200": {{
              "description": "사용자 정보 수정 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "delete": {{
          "summary": "사용자 삭제",
          "description": "사용자를 삭제합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 삭제 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "message": {{
                          "type": "string",
                          "example": "사용자가 성공적으로 삭제되었습니다"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }}
      }},
      "components": {{
      "schemas": {{
          "User": {{
          "type": "object",
          "properties": {{
              "id": {{
              "type": "integer",
              "example": 1
              }},
              "username": {{
              "type": "string",
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "createdAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }},
              "updatedAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }}
          }}
          }},
          "CreateUserRequest": {{
          "type": "object",
          "required": ["username", "email"],
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "password": {{
              "type": "string",
              "minLength": 8,
              "example": "password123"
              }}
          }}
          }},
          "UpdateUserRequest": {{
          "type": "object",
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe_updated"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john_updated@example.com"
              }}
          }}
          }},
          "Pagination": {{
          "type": "object",
          "properties": {{
              "currentPage": {{
              "type": "integer",
              "example": 1
              }},
              "totalPages": {{
              "type": "integer",
              "example": 10
              }},
              "totalItems": {{
              "type": "integer",
              "example": 100
              }},
              "itemsPerPage": {{
              "type": "integer",
              "example": 10
              }}
          }}
          }},
          "Error": {{
          "type": "object",
          "properties": {{
              "status": {{
              "type": "string",
              "example": "error"
              }},
              "message": {{
              "type": "string",
              "example": "오류가 발생했습니다"
              }},
              "code": {{
              "type": "string",
              "example": "VALIDATION_ERROR"
              }}
          }}
          }}
      }},
      "securitySchemes": {{
          "bearerAuth": {{
          "type": "http",
          "scheme": "bearer",
          "bearerFormat": "JWT"
          }}
      }}
      }},
      "security": [
      {{
          "bearerAuth": []
      }}
      ]
  }}
  }}

  각 API 엔드포인트의 상세 정보(HTTP 메서드, 경로, 요청/응답 스키마, 상태 코드)를 모두 포함하세요.
  프로젝트 특성에 맞는 추가적인 엔드포인트나 기능이 필요하면 포함하여 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  OpenAPI 3.0 표준을 따르며, 실제 구현 가능한 수준의 상세한 명세서를 작성하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    try :
      data = json.loads(answer.content)
      API_lst.append(data)
      print(f'API 명세서의 {i}번 진행중입니다.')
    except :
      print(f'{i}번쨰 오류발생')
      API_lst.append(None)
      continue

  return API_lst

def ERD_data(df):

  entities_lst = []
  relationships_lst = []

  sys_template = """
  당신은 사용자가 입력한 문장의 개요를 예상 ERD를 작성해주는 프로그램입니다. 
  주제에 대해서 예상되는 ERD를 json 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
    "entities": [
      {{
        "name": "User",
        "attributes": [
          {{
            "name": "user_id",
            "data_type": "INTEGER",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "username",
            "data_type": "VARCHAR(50)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "email",
            "data_type": "VARCHAR(100)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": true
          }}
        ]
      }}
    ],
    "relationships": [
      {{
        "from": "User",
        "to": "Order",
        "type": "one-to-many",
        "foreign_key": "user_id",
        "constraint_name": "fk_order_user"
      }}
    ]
  }}

  각 속성의 상세 정보(PK, FK, NULL 허용 여부, 데이터 타입)를 모두 포함하세요.
  제한된 방법 이외의 더 포함할 수 있는 자료가 있으면 같이 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    data = json.loads(answer.content)
    entities_lst.append(data['entities'])
    relationships_lst.append(data['relationships'])
    print(f'ERD의 {i}번 진행중입니다.')
    
  return entities_lst, relationships_lst 

In [4]:
# # 내용 요약
# contents_lst = contents_data(df)
# dataset['contents'] = contents_lst
# dataset.to_csv("../data/contents_data.csv")

In [5]:
# # ERD 생성성
# entities_lst, relationships_lst = ERD_data(df)
# dataset['relationships_data'] = relationships_lst
# dataset['ERD_data'] = entities_lst
# dataset.to_csv("../data/ERD_data.csv")

In [6]:
# # API 명세서 작성
# api_specs_lst = API_specs_data(df)
# dataset['API_specs_data'] = api_specs_lst
# dataset.to_csv("../data/api_data.csv")

## 확인결과 일부 API는 자료가 안들어가는 현상이 있어서 다시 진행

In [7]:
df = pd.read_csv("../data/api_data.csv").drop(columns='Unnamed: 0')
df.head()

,text,contents,relationships_data,ERD_data,API_specs_data
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","{'project_summary': {'title': 'GQ 서비스', 'categ...","[{'from': 'User', 'to': 'StudyGroup', 'type': ...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"{'project_summary': {'title': '기억박물관', 'catego...","[{'from': 'User', 'to': 'Photo', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...",{'project_summary': {'title': '보행 약자를 위한 지하철 길...,"[{'from': 'User', 'to': 'Route', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","{'project_summary': {'title': '1인 가구 맞춤형 플랫폼',...","[{'from': 'User', 'to': 'Post', 'type': 'one-t...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...",{'project_summary': {'title': '응급실 및 병원 찾기 웹앱'...,"[{'from': 'User', 'to': 'EmergencyRoom', 'type...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in..."


In [8]:
# 결측값 확인
df_isna = df[df['API_specs_data'].isna()]
df_isna.head()

,text,contents,relationships_data,ERD_data,API_specs_data
5,이 웹사이트는 나만의 퍼스널 쇼퍼 역할을 하는 혁신적인 쇼핑몰입니다. 사용자의 얼굴...,"{'project_summary': {'title': '퍼스널 쇼퍼 쇼핑몰', 'c...","[{'from': 'User', 'to': 'Recommendation', 'typ...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
7,"이 어플은 취준생들을 위한 종합 커뮤니티 플랫폼으로, 합격자들의 다양한 경험담과 실...",{'project_summary': {'title': '취준생 종합 커뮤니티 플랫폼...,"[{'from': 'User', 'to': 'Experience', 'type': ...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
9,이 웹앱 프로젝트는 한국어 학습자들을 위한 통합 학습 플랫폼입니다. 사용자는 일반 ...,"{'project_summary': {'title': '한국어 학습 통합 플랫폼',...","[{'from': 'User', 'to': 'Translation', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
11,"스마트 러닝 메이트는 AI 챗봇과 함께 자기주도적 학습을 체계적으로 관리하고, 집중...","{'project_summary': {'title': '스마트 러닝 메이트', 'c...","[{'from': 'User', 'to': 'LearningGoal', 'type'...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
18,"*머니메이트(MoneyMate)**는 사용자의 소비 패턴을 분석하고, 맞춤형 금융상...",{'project_summary': {'title': '머니메이트(MoneyMate...,"[{'from': 'User', 'to': 'Transaction', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN


In [ ]:
def API_specs_data_4o(df):

  API_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 기반으로 예상되는 API 명세서를 작성해주는 프로그램입니다.
  주제에 대해서 예상되는 RESTful API 명세서를 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
  "api_specification": {{
      "openapi": "3.0.0",
      "info": {{
      "title": "User Management API",
      "version": "1.0.0",
      "description": "사용자 관리를 위한 API"
      }},
      "servers": [
      {{
          "url": "https://api.example.com/v1",
          "description": "Production server"
      }}
      ],
      "paths": {{
      "/users": {{
          "get": {{
          "summary": "사용자 목록 조회",
          "description": "등록된 모든 사용자 목록을 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "page",
              "in": "query",
              "description": "페이지 번호",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 1
              }}
              }},
              {{
              "name": "limit",
              "in": "query", 
              "description": "페이지당 항목 수",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 10
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 목록 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "type": "array",
                          "items": {{
                          "$ref": "#/components/schemas/User"
                          }}
                      }},
                      "pagination": {{
                          "$ref": "#/components/schemas/Pagination"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "post": {{
          "summary": "새 사용자 생성",
          "description": "새로운 사용자를 생성합니다",
          "tags": ["Users"],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/CreateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "201": {{
              "description": "사용자 생성 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청 데이터",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }},
      "/users/{{userId}}": {{
          "get": {{
          "summary": "특정 사용자 조회",
          "description": "사용자 ID로 특정 사용자 정보를 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "put": {{
          "summary": "사용자 정보 수정",
          "description": "사용자 정보를 수정합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/UpdateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "200": {{
              "description": "사용자 정보 수정 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "delete": {{
          "summary": "사용자 삭제",
          "description": "사용자를 삭제합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 삭제 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "message": {{
                          "type": "string",
                          "example": "사용자가 성공적으로 삭제되었습니다"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }}
      }},
      "components": {{
      "schemas": {{
          "User": {{
          "type": "object",
          "properties": {{
              "id": {{
              "type": "integer",
              "example": 1
              }},
              "username": {{
              "type": "string",
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "createdAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }},
              "updatedAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }}
          }}
          }},
          "CreateUserRequest": {{
          "type": "object",
          "required": ["username", "email"],
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "password": {{
              "type": "string",
              "minLength": 8,
              "example": "password123"
              }}
          }}
          }},
          "UpdateUserRequest": {{
          "type": "object",
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe_updated"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john_updated@example.com"
              }}
          }}
          }},
          "Pagination": {{
          "type": "object",
          "properties": {{
              "currentPage": {{
              "type": "integer",
              "example": 1
              }},
              "totalPages": {{
              "type": "integer",
              "example": 10
              }},
              "totalItems": {{
              "type": "integer",
              "example": 100
              }},
              "itemsPerPage": {{
              "type": "integer",
              "example": 10
              }}
          }}
          }},
          "Error": {{
          "type": "object",
          "properties": {{
              "status": {{
              "type": "string",
              "example": "error"
              }},
              "message": {{
              "type": "string",
              "example": "오류가 발생했습니다"
              }},
              "code": {{
              "type": "string",
              "example": "VALIDATION_ERROR"
              }}
          }}
          }}
      }},
      "securitySchemes": {{
          "bearerAuth": {{
          "type": "http",
          "scheme": "bearer",
          "bearerFormat": "JWT"
          }}
      }}
      }},
      "security": [
      {{
          "bearerAuth": []
      }}
      ]
  }}
  }}

  각 API 엔드포인트의 상세 정보(HTTP 메서드, 경로, 요청/응답 스키마, 상태 코드)를 모두 포함하세요.
  프로젝트 특성에 맞는 추가적인 엔드포인트나 기능이 필요하면 포함하여 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  OpenAPI 3.0 표준을 따르며, 실제 구현 가능한 수준의 상세한 명세서를 작성하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    try :
      data = json.loads(answer.content)
      API_lst.append(data)
      print(f'API 명세서의 {i}번 진행중입니다.')
    except :
      print(f'{i}번쨰 오류발생')
      API_lst.append(None)
      continue

  return API_lst

In [9]:
API_specs_data_lst = API_specs_data(df_isna)
df_isna['API_specs_data'] = API_specs_data_lst

API 명세서의 0번 진행중입니다.
API 명세서의 1번 진행중입니다.
2번쨰 오류발생
API 명세서의 3번 진행중입니다.
4번쨰 오류발생
API 명세서의 5번 진행중입니다.
6번쨰 오류발생
API 명세서의 7번 진행중입니다.
API 명세서의 8번 진행중입니다.
API 명세서의 9번 진행중입니다.
API 명세서의 10번 진행중입니다.
11번쨰 오류발생
12번쨰 오류발생
API 명세서의 13번 진행중입니다.
API 명세서의 14번 진행중입니다.
API 명세서의 15번 진행중입니다.
API 명세서의 16번 진행중입니다.
API 명세서의 17번 진행중입니다.
API 명세서의 18번 진행중입니다.
API 명세서의 19번 진행중입니다.
20번쨰 오류발생
API 명세서의 21번 진행중입니다.
API 명세서의 22번 진행중입니다.


C:\Users\mir96\AppData\Local\Temp\ipykernel_14660\561635416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isna['API_specs_data'] = API_specs_data_lst


In [18]:
api_specs_series = pd.Series(API_specs_data_lst, index=df_isna.index)
df['API_specs_data'] = df['API_specs_data'].fillna(api_specs_series)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   text                100 non-null    object
 1   contents            100 non-null    object
 2   relationships_data  100 non-null    object
 3   ERD_data            100 non-null    object
 4   API_specs_data      94 non-null     object
dtypes: object(5)
memory usage: 4.0+ KB


In [23]:
df_isna_new = df[df['API_specs_data'].isna()]
df_isna_new.head()

,text,contents,relationships_data,ERD_data,API_specs_data
9,이 웹앱 프로젝트는 한국어 학습자들을 위한 통합 학습 플랫폼입니다. 사용자는 일반 ...,"{'project_summary': {'title': '한국어 학습 통합 플랫폼',...","[{'from': 'User', 'to': 'Translation', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",None
18,"*머니메이트(MoneyMate)**는 사용자의 소비 패턴을 분석하고, 맞춤형 금융상...",{'project_summary': {'title': '머니메이트(MoneyMate...,"[{'from': 'User', 'to': 'Transaction', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",None
21,ClauseGuard는 AI가 계약서를 분석하여 누락된 핵심 조항이나 불공정한 내용...,"{'project_summary': {'title': 'ClauseGuard', '...","[{'from': 'User', 'to': 'Contract', 'type': 'o...","[{'name': 'Contract', 'attributes': [{'name': ...",None
38,이상 거래 탐지 시스템 개발. 개요로는 카드 거래 데이터를 기반으로\r\n이상 거래...,"{'project_summary': {'title': '이상 거래 탐지 시스템', ...","[{'from': 'User', 'to': 'Transaction', 'type':...","[{'name': 'Transaction', 'attributes': [{'name...",None
40,제품 이상 판별 AI 모델 개발 . 개요로는 공장 제품 이미지 또는 데이터 기반으로...,{'project_summary': {'title': '제품 이상 판별 AI 모델 ...,"[{'from': 'Product', 'to': 'Model', 'type': 'o...","[{'name': 'Product', 'attributes': [{'name': '...",None


In [24]:
API_specs_data_lst = API_specs_data(df_isna_new)
df_isna_new['API_specs_data'] = API_specs_data_lst

API 명세서의 0번 진행중입니다.
1번쨰 오류발생
API 명세서의 2번 진행중입니다.
3번쨰 오류발생
4번쨰 오류발생
API 명세서의 5번 진행중입니다.


C:\Users\mir96\AppData\Local\Temp\ipykernel_14660\623034695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isna_new['API_specs_data'] = API_specs_data_lst


In [42]:
import re

def extract_json_from_response(content):
    """LLM 응답에서 JSON 부분만 추출"""
    # ```json ... ``` 형태에서 JSON 부분만 추출
    json_match = re.search(r'```json\s*(.*?)\s*```', content, re.DOTALL)
    if json_match:
        return json_match.group(1).strip()
    
    # 백틱 없이 { }로 둘러싸인 JSON 찾기
    json_match = re.search(r'\{.*\}', content, re.DOTALL)
    if json_match:
        return json_match.group(0).strip()
    
    # 아무것도 찾지 못했으면 원본 반환
    return content.strip()

def API_specs_data_4o(df):

  API_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 기반으로 예상되는 API 명세서를 작성해주는 프로그램입니다.
  주제에 대해서 예상되는 RESTful API 명세서를 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
  "api_specification": {{
      "openapi": "3.0.0",
      "info": {{
      "title": "User Management API",
      "version": "1.0.0",
      "description": "사용자 관리를 위한 API"
      }},
      "servers": [
      {{
          "url": "https://api.example.com/v1",
          "description": "Production server"
      }}
      ],
      "paths": {{
      "/users": {{
          "get": {{
          "summary": "사용자 목록 조회",
          "description": "등록된 모든 사용자 목록을 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "page",
              "in": "query",
              "description": "페이지 번호",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 1
              }}
              }},
              {{
              "name": "limit",
              "in": "query", 
              "description": "페이지당 항목 수",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 10
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 목록 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "type": "array",
                          "items": {{
                          "$ref": "#/components/schemas/User"
                          }}
                      }},
                      "pagination": {{
                          "$ref": "#/components/schemas/Pagination"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "post": {{
          "summary": "새 사용자 생성",
          "description": "새로운 사용자를 생성합니다",
          "tags": ["Users"],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/CreateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "201": {{
              "description": "사용자 생성 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청 데이터",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }},
      "/users/{{userId}}": {{
          "get": {{
          "summary": "특정 사용자 조회",
          "description": "사용자 ID로 특정 사용자 정보를 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "put": {{
          "summary": "사용자 정보 수정",
          "description": "사용자 정보를 수정합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/UpdateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "200": {{
              "description": "사용자 정보 수정 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "delete": {{
          "summary": "사용자 삭제",
          "description": "사용자를 삭제합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 삭제 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "message": {{
                          "type": "string",
                          "example": "사용자가 성공적으로 삭제되었습니다"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }}
      }},
      "components": {{
      "schemas": {{
          "User": {{
          "type": "object",
          "properties": {{
              "id": {{
              "type": "integer",
              "example": 1
              }},
              "username": {{
              "type": "string",
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "createdAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }},
              "updatedAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }}
          }}
          }},
          "CreateUserRequest": {{
          "type": "object",
          "required": ["username", "email"],
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "password": {{
              "type": "string",
              "minLength": 8,
              "example": "password123"
              }}
          }}
          }},
          "UpdateUserRequest": {{
          "type": "object",
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe_updated"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john_updated@example.com"
              }}
          }}
          }},
          "Pagination": {{
          "type": "object",
          "properties": {{
              "currentPage": {{
              "type": "integer",
              "example": 1
              }},
              "totalPages": {{
              "type": "integer",
              "example": 10
              }},
              "totalItems": {{
              "type": "integer",
              "example": 100
              }},
              "itemsPerPage": {{
              "type": "integer",
              "example": 10
              }}
          }}
          }},
          "Error": {{
          "type": "object",
          "properties": {{
              "status": {{
              "type": "string",
              "example": "error"
              }},
              "message": {{
              "type": "string",
              "example": "오류가 발생했습니다"
              }},
              "code": {{
              "type": "string",
              "example": "VALIDATION_ERROR"
              }}
          }}
          }}
      }},
      "securitySchemes": {{
          "bearerAuth": {{
          "type": "http",
          "scheme": "bearer",
          "bearerFormat": "JWT"
          }}
      }}
      }},
      "security": [
      {{
          "bearerAuth": []
      }}
      ]
  }}
  }}

  각 API 엔드포인트의 상세 정보(HTTP 메서드, 경로, 요청/응답 스키마, 상태 코드)를 모두 포함하세요.
  프로젝트 특성에 맞는 추가적인 엔드포인트나 기능이 필요하면 포함하여 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  OpenAPI 3.0 표준을 따르며, 실제 구현 가능한 수준의 상세한 명세서를 작성하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o',
      temperature=0,
      max_tokens = 5000
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    try :
      data = json.loads(answer.content)
      API_lst.append(data)
      print(f'API 명세서의 {i}번 진행중입니다.')
    except :
      print(f'{i}번쨰 오류발생')
      clean_json = extract_json_from_response(answer.content)
      data = json.loads(clean_json)
      API_lst.append(data)
      print(f'{i}오류 수정 완료')

  return API_lst

In [33]:
api_specs_series = pd.Series(API_specs_data_lst, index=df_isna_new.index)
df['API_specs_data'] = df['API_specs_data'].fillna(api_specs_series)
df_isna_new = df[df['API_specs_data'].isna()]
df_isna_new.head()

ValueError: Length of values (6) does not match length of index (3)

In [43]:
API_specs_data_lst = API_specs_data_4o(df_isna_new)

0번쨰 오류발생
0오류 수정 완료
1번쨰 오류발생
1오류 수정 완료
2번쨰 오류발생
2오류 수정 완료


In [45]:
api_specs_series = pd.Series(API_specs_data_lst, index=df_isna_new.index)
df['API_specs_data'] = df['API_specs_data'].fillna(api_specs_series)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   text                100 non-null    object
 1   contents            100 non-null    object
 2   relationships_data  100 non-null    object
 3   ERD_data            100 non-null    object
 4   API_specs_data      100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [52]:
df.to_csv("../data/Fine-tuning Data_preprocess-EUC.csv", encoding = 'EUC-KR')